# Data Cleaning and Aggregation

**Objective**: Collapse `whl_2025.csv` from shift-level to game-level and engineer features for ranking models.

## 1. Init & Load

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting random seed for reproducibility if needed
np.random.seed(42)

# Load Data
file_path = 'd:/A/Warton/Data/whl_2025.csv'
df = pd.read_csv(file_path)

print(f"Loaded dataset with {len(df)} records.")
df.head()

Loaded dataset with 25827 records.


,game_id,record_id,home_team,away_team,went_ot,home_off_line,home_def_pairing,away_off_line,away_def_pairing,home_goalie,...,home_goals,away_assists,away_shots,away_xg,away_max_xg,away_goals,home_penalties_committed,home_penalty_minutes,away_penalties_committed,away_penalty_minutes
0,game_1,record_1,thailand,pakistan,0,PP_kill_dwn,PP_kill_dwn,PP_up,PP_up,player_id_142,...,0,2,9,1.4645,0.2166,1,7,14,1,2
1,game_1,record_2,thailand,pakistan,0,second_off,second_def,second_off,second_def,player_id_142,...,0,2,1,0.0928,0.0928,1,0,0,0,0
2,game_1,record_3,thailand,pakistan,0,first_off,second_def,second_off,second_def,player_id_142,...,0,0,2,0.1880,0.0940,0,0,0,0,0
3,game_1,record_4,thailand,pakistan,0,second_off,first_def,second_off,first_def,player_id_142,...,0,0,1,0.0727,0.0727,0,0,0,0,0
4,game_1,record_5,thailand,pakistan,0,second_off,second_def,first_off,second_def,player_id_142,...,0,2,1,0.0769,0.0769,1,0,0,0,0


## 2. Data Aggregation

Grouping by `game_id` to aggregate statistics.

In [2]:
agg_rules = {
    'home_goals': 'sum',
    'away_goals': 'sum',
    'home_xg': 'sum',
    'away_xg': 'sum',
    'home_shots': 'sum',
    'away_shots': 'sum',
    'went_ot': 'max',
    'home_team': 'first',
    'away_team': 'first'
}

df_games = df.groupby('game_id').agg(agg_rules).reset_index()
print(f"Collapsed to {len(df_games)} unique games.")
df_games.head()

Collapsed to 1312 unique games.


,game_id,home_goals,away_goals,home_xg,away_xg,home_shots,away_shots,went_ot,home_team,away_team
0,game_1,1,3,2.8231,2.7516,21,24,0,thailand,pakistan
1,game_10,4,3,1.9254,3.3189,20,30,0,switzerland,kazakhstan
2,game_100,4,5,3.6712,3.0240,30,27,0,serbia,rwanda
3,game_1000,5,0,3.5905,2.5261,32,27,0,brazil,netherlands
4,game_1001,2,3,3.4592,3.7658,32,29,0,india,morocco


## 3. Feature Engineering

- **Winner**: Determine match winner.
- **Points**: Calculate league points (3-2-1-0 system).
- **Differentials**: GD and xGD.

In [3]:
# Determine Winner
df_games['winner'] = np.where(df_games['home_goals'] > df_games['away_goals'], 'Home', 'Away')

# Calculate Points (3 for Reg Win, 2 for OT Win, 1 for OT Loss, 0 for Reg Loss)

# Conditions
home_win = df_games['home_goals'] > df_games['away_goals']
away_win = df_games['away_goals'] > df_games['home_goals']
is_ot = df_games['went_ot'] == 1

# Vectorized Point Calculation
df_games['home_points'] = 0
df_games['away_points'] = 0

# Home Points
df_games.loc[home_win & ~is_ot, 'home_points'] = 3
df_games.loc[home_win & is_ot, 'home_points'] = 2
df_games.loc[away_win & is_ot, 'home_points'] = 1

# Away Points
df_games.loc[away_win & ~is_ot, 'away_points'] = 3
df_games.loc[away_win & is_ot, 'away_points'] = 2
df_games.loc[home_win & is_ot, 'away_points'] = 1

# Differentials
df_games['goal_diff'] = df_games['home_goals'] - df_games['away_goals']
df_games['xg_diff'] = df_games['home_xg'] - df_games['away_xg']

df_games[['game_id', 'home_team', 'away_team', 'home_goals', 'away_goals', 'home_points', 'away_points']].head()

,game_id,home_team,away_team,home_goals,away_goals,home_points,away_points
0,game_1,thailand,pakistan,1,3,0,3
1,game_10,switzerland,kazakhstan,4,3,3,0
2,game_100,serbia,rwanda,4,5,0,3
3,game_1000,brazil,netherlands,5,0,3,0
4,game_1001,india,morocco,2,3,0,3


## 4. Quality Control (QC)

Running assertions and statistical checks.

In [4]:
# 1. Uniqueness Check
assert df_games['game_id'].is_unique, "CRITICAL: Game IDs are not unique!"

# 2. Consistency Check (Points)
# In every game, sum of points should be 3 (Reg) or 3 (OT 2+1).
assert (df_games['home_points'] + df_games['away_points']).isin([3]).all(), "Point allocation error found!"

print("✅ Assertions Passed.")

# 3. Distribution Check
print("\nGoal Distribution:")
print(df_games[['home_goals', 'away_goals']].describe())

# Statistician's Note
print("\n--- Statistician's Commentary ---")
h_mean = df_games['home_goals'].mean()
a_mean = df_games['away_goals'].mean()
print(f"Observed Home Advantage: +{h_mean - a_mean:.3f} goals/game")
if (h_mean - a_mean) > 0.2:
    print("Note: Home advantage seems significant, typical of professional leagues.")
else:
    print("Note: Home advantage is mild or negligible.")


✅ Assertions Passed.

Goal Distribution:
        home_goals   away_goals
count  1312.000000  1312.000000
mean      3.091463     2.666159
std       1.814642     1.721621
min       0.000000     0.000000
25%       2.000000     1.000000
50%       3.000000     3.000000
75%       4.000000     4.000000
max      11.000000    10.000000

--- Statistician's Commentary ---
Observed Home Advantage: +0.425 goals/game
Note: Home advantage seems significant, typical of professional leagues.


## 5. result Display

Displaying the final aggregated dataframe.

In [5]:
df_games

,game_id,home_goals,away_goals,home_xg,away_xg,home_shots,away_shots,went_ot,home_team,away_team,winner,home_points,away_points,goal_diff,xg_diff
0,game_1,1,3,2.8231,2.7516,21,24,0,thailand,pakistan,Away,0,3,-2,0.0715
1,game_10,4,3,1.9254,3.3189,20,30,0,switzerland,kazakhstan,Home,3,0,1,-1.3935
2,game_100,4,5,3.6712,3.0240,30,27,0,serbia,rwanda,Away,0,3,-1,0.6472
3,game_1000,5,0,3.5905,2.5261,32,27,0,brazil,netherlands,Home,3,0,5,1.0644
4,game_1001,2,3,3.4592,3.7658,32,29,0,india,morocco,Away,0,3,-1,-0.3066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,game_995,4,3,2.9710,2.8999,30,23,1,vietnam,mexico,Home,2,1,1,0.0711
1308,game_996,0,5,1.2391,3.3882,12,31,0,kazakhstan,indonesia,Away,0,3,-5,-2.1491
1309,game_997,3,5,2.0455,3.6745,22,34,0,canada,south_korea,Away,0,3,-2,-1.6290
1310,game_998,4,2,3.2982,2.2124,35,21,0,switzerland,uae,Home,3,0,2,1.0858
